# Intro to Deep Learning, aka., Neural Networks

This notebook is intended as a lean, hands-on introduction to neural networks with the goal of acquiring an intuition on the reason of the fundamental components that are put together in order to any of the deep learning networks that achieve such impressive results as autonomously driving cars, beating any Go master or learning to paint as an all-time master, as the following video showcases.

I've also used partially in my Computer Science class at Dragon Academy (Toronto) as a background context in teaching programming, e.g., for motivating the use of things like different data structures, modularity, or higher-order functions.

**Note:** For best viewing experience, download this document and open it directly in your own jupyter notebook. Otherwise, if you get a weird line wrapping when reading this python notebook directly from its github page, you may try to zoom out in your browser for this page (```ctrl++```). That may specially be helpful for the diagrams drawn below. Another option is to view this notebook directly on the jupyter site
http://nbviewer.jupyter.org/github/MASantos/DeepLearning/blob/DeepLearningBook/DeepLearningBook/NN%20Intro.ipynb

[![Two minute papers: DNN learns Van gogh's art](TwoMinutesPaperDNNVanGoh.png)](https://www.youtube.com/watch?v=-R9bJGNHltQ)

# The Gist of Learning

The most essential step in learning is distinguishing two distinct groups among a given collection of objects. This is at its root a non-linear process. The very same classification we obtain entails a non-linearity: "*on one side everything is smooth and green; on the other side everything is smooth but red*". We will take it as a principle that *it is this process that is the gist of human learning and reasoning capabilities*[<sup id="ref0">+</sup>](#f0).

The simplest mathematical and computational toy model that is capable of making such distinctions is the **perceptron**. This is the most simple step which **all neural networks** build upon. It is in fact thus the "hello world" example of a neural network.

[<sup id="f0">+</sup>] The rationale for such a principle is based on the evolution of Physics and Mathematics during the XX and early XXI centuries. The classification of chemical elements, as explained by Quantum Physics, is intimately related to the atoms being invariant under certain type of transformations. This is an instance of what's called a symmetry of the system. Indeed in modern Physics, as given by Quantum Field Theory and the Standard Model, it is the symmetries of the elementary particles that determine their interactions! The classification of some elementary particles (some baryons and mesons) into families by physicist Murray Gell-Mann is an example where the concept of symmetry is used in the clustering of different objects into distinct groups. The so-called $SU(3)$ symmetry was at the heart of such classification. Furhtermore, Felix Klein, at the beginning of the XX century showed how Geometry can be derived from the symmtry transformations we consider (Euclidean geometry is determined by Affine transformations -rotations, reflections, invertions and translations-, Projective Geometry is determined by Homographies -e.g. fractional transformations). In this context, both the features that make sense to distinguish as well as the objects themselves are given by those properties (e.g., distance, angle, projection,...) and arrangments (read figures) that are invariant under certain transformations (symmetry). It is this invariance that classifies a triangle as something distinct from a square, a pentagon, an hexagon...Finally, modern foundations of Mathematics rely on the concept of *morphism*. This could be viewed as a stripped down version of a symmetry transformation. It allows the grouping of different collections mathematical objects into categories where those objects who belong to any given one are related by a morphism (think of an arrow or an edge) of a given type. For instance, the collection of all sets is *grouped* into the category called **Set** where the morphisms are simply bijective functions. Whence, the common pattern in al these cases is the intimate relation between symmetry/morphism and classifying into different groups. Finally, the Langland's Program in Mathematics, a quest for the unifying  "Theory of Everything" in mathematis is in a way a quest for the ultimate symmetries allowing such unified view of all *distinct domains* of Mathematics (Geometry, Number Theory, Harmonic Analysis, Representation theory, etc. Given that ultimately all sciences are a reflection of how human brain and intelligence work. it doesn't seem arbitrary to raise the claim that clustering is at the heart of human understanding and intelligence in general.

# Perceptron

<pre>
2 inputs                                     N inputs

X1o                                          X1o
   \                                             \
    \W1                                           \
     \             --                              \             --
      ----------  |   --------o y            X2o--------------  |   --------o y
     /          __|                          .     /          __|
    /W2                                      .    /            
X2o/                                         .   /
                                                /
         |x2                                Xno/ 
     \ + |                                    
      \  |  (w1, w2)
     - \ | /^  +
   _____\|/_________ x1
         |\
     -   | \   +
         |  \

The simple perceptron is capable of classifying points (x1, x2) (or of making distinctions) by drawing
a line. This, in effect, generates a non-linearity in our view of the universe: things on "the left" 
side of the line and things on "the right" side. The line has a perpendicular given by the vector 
(w1, w2). The latter points in the direction where the output y is larger in the algebraic sense 
(smaller if the non-linear function is an inversion). In the case of N inputs, the perceptron 
classifies points in an N-dimensional space by drawing an (N-1)-hyperplane. For N>=3, it classifies 
them between those "inside"  that hyperplane and those outside it.
</pre> 

## Non-linearity

In the above plot, the symbol denoted by
```
          __
      ___|____
       __|
  
```
represents a non-linear function. In layman terms, this is a function that is not a straight line. In other terms, at some point it shows a "jump" *relative to its value at another point* even 
though elsewhere it may run as smooth as it possibly can be.

You may raise the following concern: "*If you said that learning entails a non-linear process and you build into 
the model a non-linear function, it would seem you are cheating, as you'd be 'inserting' into the machine the 
very essencial process we expect it to perform*".

Indeed, if we would cut off the non-linear function the perceptron would loose its **"learning" capability of
_classifying_**. However, all is not lost!

In such case, the toy model reduces to
<pre>
X1o                                          
   \                                          
    \W1                                          
     \                                         
      --------------o y            
     /                                    
    /W2                                                 
X2o/                                         
</pre>
Mathematically, this model corresponds to a weighted average of both inputs. Is this _learning_? Well, it is 
as much as it is learning what we do when we obtain the average percentage of price increase over the year 
**in order to get an insight into the amount of inflation during said period**. In statistical parlance, this
corresponds to calculating the well-known estimator of central tendency called the *mean* or *average* of a 
sample. Indeed, **this number abstracts a general trend out of all the details given by all measurements in 
the sample**. 

While such a process is usually considered an example of learning[<sup id="ref1">1</sup>](#f1), it is a very limiting one. We will see that 
not being able to classify renders this model incapable of computing any general function: it can calculate 
one and only one function, namely that weighted average.

You could still reply: "*OK, but putting aside such a limiting case of learning, I think basically you are 
cheating when you include that non-linear function into your model for the reason mentioned above. Furthermore,
I've read about current cutting-edge research on computational systems built using chemical reactions or using 
DNA molecules or based on complex systems showing self-organization, etc. that [apparently] would not be using 
an ad-hoc inserted non-linearity as you do. The learning, and thus the knowledge, is an **emergent thing** that comes out of the system in these cases*".

Fair enough! I'm building that non-linearity ad-hoc into my model. Mea culpa. 

In my defense, though, the details of how a sytem in nature may have developed and implemented such a 
non-linear response is not that relevant right now. We still have the task of understanding how such general 
learning and computational capabilities can be modelled using a perceptron. In other terms, there is a long way to go and we must take it one step at a time. 

The very first such steps is indeed understanding how we can model any computable function using such a simple 
toy model of a perceptron as a building block. In layman terms, this would give us the analog of a computer.

In other words, given any specific problem the first thing we need to learn is **how to devise a neural 
network that solves my problem**. This is the gist of almost all literature on neural networks (or deep 
learning, using the new buzz word). 

One way of stating this problem is then: Given a set of input values and their possible outputs, how can I 
device the internals of a deep learning network that (1) learns that mapping out of the examples provided,  
(2) correctly applies the function learned on a new set of input values not used before during the learning 
phase, and (3) that make sensible "generalizations",i.e. gives reasonable values for cases further away from 
those used during learning and testing.

It is in this context where the details of the non-linear function I put in there become basically irrelevant. Any 
non-linear function will do the job. In everyday life, the details, however, may be important, but only
because our society puts so much emphasis in money and time: different choices of non-linear functions may make the learning process much slower/require many more examples for achieving the same accuracy, or be less acurrate given a fixed amount of available examples. That is, from a conceptual point of view, it doesn't
matter which non-linear function we use, but in practical cases, time and/or money will impose some
constraints.

<sup id="f1">[1]</sup>It's the limiting case of a [regression analysis](https://en.wikipedia.org/wiki/Regression_analysis) for when the number of independent variables is zero and the functional regression relation reduces to a constant.[^](#ref1)

### Non-linear functions

These are also called *activation functions*. Here just a few examples.

#### Step functions

For instance, the **bipolar step-function**: -1 for any strictly negative value of its arguments and +1 for the rest, including zero.
<pre>
           y
         +1|______   
           | 
     ______|______ x  
           |
      _____|-1
           |
           
</pre>



Another is the so-called **Heaviside step-function**: For any negative value of its arguments it yields 0; for all others, +1.
<pre>
           y
         +1|______   
           | 
   0 ______|______ x  
           |
           |
           
</pre>



####  The rectifier linear unit

This is a function that lets all non-negative values pass *as they are* (identity function). However, it zeros all negative values. It looks like this
<pre>
           y
           |  / y=x  
     y=0   | / 
      _____|/_____ x  
           |
           
</pre>

In [1]:
def nonLinearBipolarStep(x,string=None):
    if not string: return (-1 if x<0 else 1 )
    else: return ('-' if x<0 else '1')

def nonLinearHeaviside(x,string=None):
    if not string: return (0 if x<0 else 1 )
    else: return ('o' if x<0 else '1')

def nonLinearRelu(x,string=None):
    r = max(0,x)
    if not string: return r
    else: return str(r)

def identity(x,string=None):
    if not string: return x
    else: return str(x)

In [2]:
def perceptron(x1,x2,w1,w2,string=False,debug=False, activation=nonLinearBipolarStep):
    """
    Simple perceptron w/ 2 inputs, 2 weights and Heaviside step function
    """
    z = x1*w1+x2*w2
    if debug: print("x1: ",x1,"  x2: ",x2,"  w1: ",w1,"  w2: ",w2,"  z: ",z,end=" -->  ")
    return activation(z,string)
    """
    if not str: return -1 if z<0 else 1;
    else: return "-" if z<0 else "1";
    """
print(perceptron(-0,2,0,1))

def nPerceptron(X,W,string=False,debug=False,activation=nonLinearBipolarStep):
    assert(len(X)==len(W))
    z=0;
    for i in range(0, len(X)):
        z+=X[i]*W[i]
    return activation(z,string)

nPerceptron([3,2,2],[0,1,2])

1


1

## Classifying the plane

A single perceptron is able to draw a distinction between two parts of the plane. We will use this to build a classifications of the plane beyond that given by a simple one. These are non-linear classifications of the plane, so called right because the boundaries of the domains defined such way a not straight lines.

In order to do so we will use the simple perceptron as a kind of a Lego piece. By plugging multiple perpceptrons together we hope to be able to make more sophisticated distinctions, aka, classifications. Such a setup of multiple perceptrons has been known since the middle of the XX century by differents like Multilayer Perceptron, Neural Network or, in the new parlance, Deep Learning.

Let's define a function ```scan2InputNN``` that takes any kind of neural network with only two inputs and outputs a map of the plane with the network output at each point. This will give us a simple graphical view of how that neural network classifies the plane. 

Here the set of weights is fixed beforehand. The goal is simply to get some intuition on how the neuron divides the plane.

In [20]:
from math import floor
import numpy as np

def scan2InputNN(nn,weights,xysup=11,showHidden=None,debug=False): 
    """
    scan2InputNN(nn,weights,xysup=11,showHidden=None,debug=False)
    
    Pass a 'neural network with 2 inputs and its weights, then scan a given range of values for those inputs and see 
     where the NN classifies those points into ('-' / '+')
     
    NN needs to have defined inputs and weights as simple array. Also optional variables showHidden, 
    for showing values of hidden neurons, or debug for additional info
    
    xysup=11 #supremum of x,y ranges, with -xysup(1-[1/2]) <= x,y <= xysup(1-[1/2]). 
    E.g., xysup=11, xy \in [-5,5]
    
    It calls the NN as: nn([x1,x2],weights,showHidden=showHidden,debug=debug)
    """
    X1 = np.arange(0,xysup)
    X2 = np.arange(0,xysup)
    X1 -= floor(xysup/2)
    X2 -= floor(xysup/2)
    X1

    if showHidden: print("Showing hidden neuron ",showHidden)
        
    print("      ",end="") #formatting
    for x1 in X1:          #Print first line w/ x-coord 
        print(x1,end="  ")
    print("\n\t",end="") 
    for x2 in reversed(X2):                               #Print first y coor. (+ values top/first=>reverse array)
        print(x2,end=" ") if x2<0 else print(x2,end="  ")
        for x1 in X1:                                     #...then go on w/ values along x-axis
            #print(nn1HiddenLayer3([x1,x2],W),end="  " )
            #if debug: help(nn)
            print(nn([x1,x2],weights,showHidden=showHidden,debug=debug),end="  " )
        print("\n\t",end="")

"""def scan2InputPerceptron(ptron,weights,debug=False):
    def wraptron(X,weights,showHidden=None):
        ptron(X[0],X[1],weights[0],weights[1],string=True,debug=debug)
    
    scan2InputNN(wraptron,weights)
"""    
def wraptron(X,weights,showHidden=None,debug=False):
    """
    A wrapper around the perceptron as defined above in order to be used with scan2InputtNN
    """
    return perceptron(X[0],X[1],weights[0],weights[1],string=True,debug=debug)


In [4]:
#scan2InputPerceptron(perceptron,weights=[0,1],debug=True)
scan2InputNN(wraptron,weights=[2,3],debug=False)
print(perceptron(-4,-3,1,1,debug=True))

      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  -  1  1  1  1  1  1  1  1  1  1  
	2  -  -  1  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  -  1  1  1  1  
	-2 -  -  -  -  -  -  -  -  1  1  1  
	-3 -  -  -  -  -  -  -  -  -  -  1  
	-4 -  -  -  -  -  -  -  -  -  -  -  
	-5 -  -  -  -  -  -  -  -  -  -  -  
	x1:  -4   x2:  -3   w1:  1   w2:  1   z:  -7 -->  -1


## Turing complete: XOR gate

During the XX century the idea that reasoning could be grasped by an automatic computing machine was quite 
widespread. The SciFi literature and movies constantly pictures even in our days computing machines as 
having become a sentient being, or at least an independent reasoning being. These are the androids. 

It's still unclear what conciousness is, even less so whether we'll be able to build a machine that gains 
such. However, a key feature we would like to simulate is computational capabilities. In particular, the 
posibility of solving any problem that is computationally solvable. We need a Turing-complete machine. 

A basic neural network based on the perceptron can be shown to bear the essence of computation. Indeed, all
propositional calculus can be build on one single logica operator (gate), e.g., the XOR one. Let's see how 
we can model it.


<pre>
          W1                    __     h1
X1o--------------------------  |   -----------o
   \                /        __|               \
    \              /W2                          \
     \            /                              \ 
      \          /                                \
       \        /                                  \
        \      /                                    \w5
         \    /                                      \
          \  /                                        \
            /                                          \          __            
           /\                                          --------  |  -------o y
          /  \                                         /       __|              
         /    \                                       /
        /      \                                     /
       /        \                                   /w6
      /          \                                 /
     /            \                               /
    /              \W3                           /
   /                \                           /
  /                  \          __     h2      /
X2o--------------------------  |   -----------o
           W4                __|



        X2 |                            h2  |
           |                                |
           1  0            ====>         ?  |  ?
           |                                |
   --------0--1----- X1             ---------------- h1
           |                                |
           |                             ?  |  ?
           |                                |
           |                                |
</pre>


In [5]:

def nn1HiddenLayer2(X,W,showHidden=None,debug=False):
    assert len(W)==6 
    strings=[False,False,False,False,False]
    h=0
    if showHidden : 
        h=(showHidden-1)%2 # h1-2
        assert h>=0
        strings[h]=True 
        
    hl=[]
    hl.append(nPerceptron(X=X,W=W[0:2],string=strings[0]) )
    hl.append(nPerceptron(X=X,W=W[2:4],string=strings[1]) )

    if showHidden: return hl[h]
    
    return nPerceptron(X=[hl[0],hl[1]],W=W[4:6],string=True)

As mentioned above, this is in effect a neural network: we have used the perceptron block and connected three of these together.

If you run the scan function with the following weights, you will see that the points a(1,0), b(0,1) get mapped onto +1, while the points (0,0) and (1,1) get mapped onto -1. Thus, in effect this neural network can distinguish between them in the same way the XOR function does. It is therefore a faithful model of the XOR gate.


In [6]:
weights1Layer=[-1,1,
              1,-1,
              -1,-1
             ]
scan2InputNN(nn1HiddenLayer2,weights=weights1Layer,showHidden=None,xysup=3)

      -1  0  1  
	1  1  1  -  
	0  1  -  1  
	-1 -  1  1  
	

### Analysis

<pre>
XOR(o) = 1  |     No lime                                 No line 
XOR(x) = 0  o x   distinguish                        |    distinguish                  
            |     o from x         W               o |    o from x
      ______x_o____             ------->        _____x____
            |                                        | o          
            |                                        |             
            |                \                       | 
                              \
                               \  S ° W 
            |                   \                 \  |       Infinite lines
            | S                   \                o | x     distinguish o from x
            |                       \------>   _____\|_____  e.g., 2nd diagonal!
            v                                        |\
                                                     | o
                    No line                          |  \
            | ®     distinguish
       _____|_____  o from x
            |
            |
            
      Neither the linear transformation W nor the non-linear step function S alone allow us 
      to distinguish the two input cases of the XOR function. However, combining both yields
      a separable set.
</pre>

We can write the whole network as 

$y\,=\,S\left( \mathbf{\overrightarrow{w'}}\cdot \mathbf{\overrightarrow{h}} \right)\,=\,S\left((w_5,\,w_6)\cdot\begin{pmatrix}h_1\\h_2\end{pmatrix}\right)$  
$\mathbf{\overrightarrow{h}}\,=\,S\left(\mathbf{W}\,\mathbf{\overrightarrow{x}}\right)$

$S\big( (a,\,b)\big)\,\equiv\,\big((S(a),\,S(b))\big)$ and $S(a)\,=\,\left\{\begin{array} \,-1\;\mbox{if a<0}\\1\;\mbox{otherwise}\end{array}\right.$   
<pre>
        |         Properties of S:     \  |  / Leaves invariant both diagonal
       1|__ S(a)                        \ | /            |       
   _____|______                     _____\|/______     x | +  Maps origin and positive base vec -> (1,1)
      __|-1                              /|\         __^_o_^__  
        |                               / | \            |›x  Maps negative base vec onto 2nd diagonal  
        |                              /  |  \           | 
            
</pre>
                                                 
with function $S$ being the (non-linear, Heaviside) step function. Also,

$\mathbf{W}\,\mathbf{\overrightarrow{x}}\,=\,\begin{bmatrix}-1 & 1 \\1 & -1\end{bmatrix}\,\begin{pmatrix}x_1\\x_2\end{pmatrix} $

and 

$\mathbf{\overrightarrow{x}}\,=\,\left\{\begin{array}\,(1,0)^t \\ (0,1)^t\end{array}\right. \quad\Rightarrow\quad W\mathbf{\overrightarrow{x}}\,=\,\left\{\begin{array}\,-(1,-1)^t\\-(-1,1)^t\end{array}\right.$

$\mathbf{\overrightarrow{x}}\,=\,\left\{\begin{array}\,(1,1)^t \\ (0,0)^t\end{array}\right. \quad\Rightarrow\quad W\mathbf{\overrightarrow{x}}\,=\,(0,0)$

Thus, the linear transformation $\mathbf{W}$ maps all four inputs **interspersed** along the second diagonal. On the other hand, if we'd feed the input directly to the non-linear step function $S$, all four input values would be mapped onto the single point $(1,1)$. Whence, in either case a projection onto any direction will not be able to untangle the two input cases of the XOR function.

However, it works if we combine both! Let's see how. First $\mathbf{W}$ maps the input cases onto the second diagonal and then the step function $S$ pulls the origin $(0,0$ off that diagonal and maps it onto point $(1,1)$. A straight line can then be used to distinguish them.


**Question: Which line could we then use to actually classify the two input cases of the XOR function?** Ans: There is of course an infinite number of lines that do the job. Among them we have, e.g., the 2nd diagonal $y=-x$, but also any othe parallel to that cuts through the first quadrant but leaves point $(1,1)$ still on its right side, i.e., $y=-x+b$ with $0\lt\,b\,\lt 2$<sup id="ref2">[2](#f2)</sup>.

**Remark: Modelling the XOR function entails classifying a set of points that can *not* be classified by just drawing a straight, dividing line on the plane. We achieved that despite the fact the our basic building block, the perceptron, is only capable of making a linear classification, i.e., based on a line!**.


<sup id="f2">[2]</sup>Usually in the literature the perceptron is defined including a constant terms $b$. In the Machine learning community this is known as the bias; in physics, however, it's called the external field. Here, we haven't mentioned so far such a term. We will see later how it can be introduced in a natural way by simply building on the freedom we have for setting up learning/classifying machines.[^](#ref2)


<!--
#### Geometrical interpretation
The linear transformation given by $\mathbf{W}$ can be seen as $\mathbb{-1}\,+\,\mathbf{\sigma_x}$, where $\sigma_x\,=\,\begin{bmatrix}0&1\\1&0\end{bmatrix}\,=\,R_{-\pi/2}\cdot m_y$, i.e., a reflection over the y-axis $m_y$ (inverting the orientation of the x-axis) followed by a clock-wise rotation of $90\deg$, $R_{-\pi/2}$. Furthermore, the first term of $\mathbf{W}$, which corresponds to an inversion over the origin, can be constructed as a reflection over $y$-axis followed by another over the $x$-axis, i.e., $\mathbb{-1}\,=\,m_x\,m_y$. Whence, it is
$$\mathbf{W}\,=\,\left[m_x\,+\,R_{-\pi/2}\right]\,m_y$$

This leads to  the mapping $\overrightarrow{x}\,\to\,-\overrightarrow{x}\,+\,\overrightarrow{x'}$ where $\overrightarrow{x'}\,=\,(x_2,x_1)$. In terms of the reflections, $\overrightarrow{x}\,\to\,-\,(1\,-\,R_{-\pi/2}\,m_y)\,\overrightarrow{x}$

-->

### Multilayer perceptron, aka., NN

Some more examples of non-linear classificationsc

<pre>
          W1                           __     h1
X1o---------------------------------  |   ------------o
   \ \              /               __|                \
    \  \           /W2                                  \
     \   \        /                                      \ 
      \    \     /                                        \
       \     \  /                                          \
        \      /\                                           \w7
         \    /  \ W3                                        \
          \  /    \                                           \
            /      \                    __     h2          w8  \           __            
           /\       /----------------  |   ------------o----------------  |   --------o y
          /  \     /                 __|                       /        __|              
         /    \   /W4                                         /
        /      \ /                                           /
       /      / \                                           /w9
      /     /    \                                         /
     /    /       \                                       /
    /   /          \W5                                   /
   /  /             \                                   /
  / /                \                  __     h3      /
X2o----------------------------------  |   ------------o
           W6                        __|
</pre>


In [24]:
from math import *
import numpy as np

def nn1HiddenLayer3(X,W,showHidden=None,debug=False):
    z1 = X[0]*W[0]+X[1]*W[1];
    z2 = X[0]*W[2]+X[1]*W[3];
    z3 = X[0]*W[4]+X[1]*W[5];

    h1 = -1 if z1<0 else 1;
    h2 = -1 if z2<0 else 1;
    h3 = -1 if z3<0 else 1;
  
    z  = h1*W[6]+h2*W[7]+h3*W[8];
    return "-" if z<0 else "1";
    
def ng_nn1HiddenLayer3(X,W,string=False,debug=False,activation=nonLinearBipolarStep,showHidden=None):
    h1 = perceptron(X[0],X[1],W[0],W[1],string=False,debug=debug,activation=activation)
    h2 = perceptron(X[0],X[1],W[2],W[3],string=False,debug=debug,activation=activation)
    h3 = perceptron(X[0],X[1],W[4],W[5],string=False,debug=debug,activation=activation)
    
    return nPerceptron([h1,h2,h3],[W[6],W[7],W[8]],string=str,debug=debug,activation=activation) ;

weights1HiddenLayer3 = [0,1,
     1,1,
     2,3,
     1,-2,1]

len(weights1HiddenLayer3)

9

In [22]:
scan2InputNN(ng_nn1HiddenLayer3,weights1HiddenLayer3)

      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  1  1  1  1  1  1  1  1  1  1  1  
	2  1  1  1  1  1  1  1  1  1  1  1  
	1  1  1  1  1  1  1  1  1  1  1  1  
	0  1  1  1  1  1  1  1  1  1  1  1  
	-1 1  1  1  1  1  1  -  -  -  -  -  
	-2 1  1  1  1  1  1  1  -  -  -  -  
	-3 1  1  1  1  1  1  1  1  -  -  -  
	-4 1  1  1  1  1  1  1  1  1  -  -  
	-5 1  1  1  1  1  1  1  1  1  1  -  
	



<pre>
          W1                           __     h1
X1o---------------------------------  |   ------------o
   \ \              /               __|                \
    \  \           /W2                                  \
     \   \        /                                      \ 
      \    \     /                                        \
       \     \  /                                          \
        \      /\                                           \w7
         \    /  \ W3                                        \
          \  /    \                                           \
            /      \                    __     h2          w8  \           __            
           /\       /----------------  |   ------------o----------------  |   --------o y
          /  \     /                 __|                       /        __|              
         /    \   /W4                                         /
        /      \ /                                           /
       /      / \                                           /w9
      /     /    \                                         /
     /    /       \                                       /
    /   /          \W5                                   /
   /  /             \                                   /
  / /                \                  __     h3      /
X2o----------------------------------  |   ------------o
           W6                        __|
</pre>


In [29]:
def nn1HiddenLayer3B(X,W,showHidden=None,debug=False):
    """
    The same NN with 1 hidden layer of 3 neurons.
    
    Rewritten here for implementing visualizing each internal neuron separately.
    """
    strings=[False,False,False]
    h=0
    if showHidden : 
        h=showHidden-1
        assert h>=0
        strings[h]=True 
        
    hl=[]
    hl.append( perceptron(X[0],X[1],W[0],W[1],strings[0]) )
    hl.append( perceptron(X[0],X[1],W[2],W[3],strings[1]) )
    hl.append( perceptron(X[0],X[1],W[4],W[5],strings[2]) )
    
    if debug: print("h1: ",hl[0],"  h2: ",hl[1],"  h3: ",hl[2],"\nh: ",h,"  showHidden: ",showHidden)
        
    if showHidden: return hl[h]
        
    z = hl[0]*W[6]+hl[1]*W[7]
    
    if debug: print("S^{-1}(y): ",z+hl[2]*W[8],"\nW7 / W8 / W9 : ",W[6],W[7],W[8])
        
    return perceptron(z,hl[2],1,W[8],string=True)

weights1HiddenLayer3B = [0,1,
     1,1,
     2,3,
     1,2,-3]

scan2InputNN(ng_nn1HiddenLayer3,weights1HiddenLayer3B,xysup=21,showHidden=None)

      -10  -9  -8  -7  -6  -5  -4  -3  -2  -1  0  1  2  3  4  5  6  7  8  9  10  
	10  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	9  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	8  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	7  -  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	6  1  -  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	5  1  1  1  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  -  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	3  1  1  1  1  1  1  -  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	2  1  1  1  1  1  1  1  -  1  1  1  1  1  1  1  1  1  1  1  1  1  
	1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  
	-1 1  1  1  1  1  1  1  1  1  1  1  1  -  -  -  -  -  -  -  -  -  
	-2 1  1  1  1  1  1  1  1  1  1  1  1  1  -  -  -  -  -  -  -  -  
	-3 1  1  1  1  1  1  1  1  1  1 

In [19]:
nn1HiddenLayer3B([3,2],weights1HiddenLayer3,showHidden=None,debug=True)

h1:  1   h2:  1   h3:  1 
h:  0   showHidden:  None
S^{-1}(y):  0 
W7 / W8 / W9 :  1 -2 1


'1'

#### 3 Layers NN
<pre>
          W1                           __     h1
X1o---------------------------------  |   ------o
   \ \              /               __|          \
    \  \           /W2                            \w7
     \   \        /                                \ 
      \    \     /                                  \            __       J1
       \     \  /                                    /--------  |   ------o
        \      /\                                   /         __|          \
         \    /  \ W3                              /w8                      \w11
          \  /    \                               /                          \
            /      \                    __     h2/                            \        __            
           /\       /----------------  |   -----o  ----------/     /----------------  |   ------------o y
          /  \     /                 __|         \                             /    __|              
         /    \   /W4                             \w9                         /
        /      \ /                                 \                         /w12
       /      / \                                   \             __        /
      /     /    \                                   /---------  |   ------o
     /    /       \                                 /          __|         J2
    /   /          \W5                             /w10       
   /  /             \                             /      
  / /                \                  __     h3/      
X2o----------------------------------  |   -----o
           W6                        __|
</pre>

In [12]:
def nn2HiddenLayer32(X,W,showHidden=None,debug=False):
    assert len(W)==12 
    strings=[False,False,False,False,False]
    h=0
    if showHidden : 
        h=showHidden-1 # h1-3: 0-2 ; j1-2:3,4
        assert h>=0 
        strings[h]=True 
        
    hl=[]
    hl.append(nPerceptron(X=X,W=W[0:2],string=strings[0]) )
    hl.append(nPerceptron(X=X,W=W[2:4],string=strings[1]) )
    hl.append(nPerceptron(X=X,W=W[4:6],string=strings[2]) )
    
    if showHidden and h<3: return hl[h]
    
    jl=[]
    jl.append(nPerceptron(X=[hl[0],hl[1]],W=W[6:8],string=strings[3]) )
    jl.append(nPerceptron(X=[hl[1],hl[2]],W=W[8:10],string=strings[4]) )

    if showHidden: return jl[h-3]  #only happens if not printing first hidden layer hl[h]

    if debug:
        for i in range(5):
            if i<3:
                v = hl[i]
                idx = i
                lbl='h'
            else:
                v = jl[i-3]
                idx = i-3
                lbl='J'
            print(lbl,"",idx,": ",v,"  ",end="",sep="")
        print("\nh: ",h,"  showHidden: ",showHidden,sep="")

    if showHidden: return hl[h] if h<3 else jl[h-3]
    
    return nPerceptron(X=[jl[0],jl[1]],W=W[10:12],string=True)


In [13]:
weights2Layers=[0,1,       #w1,w2
                1,1,       #w3,w4
                2,3,       #w5,w6
                -0.5,-.2,-.1,0.5, #w7,w8,w9,w10
                1,1]       #w11,12

#len([1,23,3])
#len(weights2Layers)
#nn2HiddenLayer32([3,3],weights2Layers)
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=None,xysup=11)

      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  -  1  1  1  1  1  1  1  1  1  1  
	2  -  -  1  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 1  1  1  1  1  1  1  1  1  1  1  
	-2 1  1  1  1  1  1  1  1  1  1  1  
	-3 1  1  1  1  1  1  1  1  1  1  1  
	-4 1  1  1  1  1  1  1  1  1  1  1  
	-5 1  1  1  1  1  1  1  1  1  1  1  
	

## How?

Let's see how each hidden layer contributes to achieve the above classification of the plane.

In [14]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=1,xysup=11)

Showing hidden neuron  1
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  1  1  1  1  1  1  1  1  1  1  1  
	2  1  1  1  1  1  1  1  1  1  1  1  
	1  1  1  1  1  1  1  1  1  1  1  1  
	0  1  1  1  1  1  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  -  -  -  -  -  
	-2 -  -  -  -  -  -  -  -  -  -  -  
	-3 -  -  -  -  -  -  -  -  -  -  -  
	-4 -  -  -  -  -  -  -  -  -  -  -  
	-5 -  -  -  -  -  -  -  -  -  -  -  
	

In [15]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=2,xysup=11)

Showing hidden neuron  2
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  -  1  1  1  1  1  1  1  1  1  1  
	3  -  -  1  1  1  1  1  1  1  1  1  
	2  -  -  -  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  1  1  1  1  1  
	-2 -  -  -  -  -  -  -  1  1  1  1  
	-3 -  -  -  -  -  -  -  -  1  1  1  
	-4 -  -  -  -  -  -  -  -  -  1  1  
	-5 -  -  -  -  -  -  -  -  -  -  1  
	

In [16]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=3,xysup=11)

Showing hidden neuron  3
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  -  1  1  1  1  1  1  1  1  1  1  
	2  -  -  1  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  -  1  1  1  1  
	-2 -  -  -  -  -  -  -  -  1  1  1  
	-3 -  -  -  -  -  -  -  -  -  -  1  
	-4 -  -  -  -  -  -  -  -  -  -  -  
	-5 -  -  -  -  -  -  -  -  -  -  -  
	

In [17]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=4,xysup=11)

Showing hidden neuron  4
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  -  -  -  -  -  -  -  -  -  -  -  
	4  -  -  -  -  -  -  -  -  -  -  -  
	3  -  -  -  -  -  -  -  -  -  -  -  
	2  -  -  -  -  -  -  -  -  -  -  -  
	1  -  -  -  -  -  -  -  -  -  -  -  
	0  -  -  -  -  -  -  -  -  -  -  -  
	-1 1  1  1  1  1  1  1  1  1  1  1  
	-2 1  1  1  1  1  1  1  1  1  1  1  
	-3 1  1  1  1  1  1  1  1  1  1  1  
	-4 1  1  1  1  1  1  1  1  1  1  1  
	-5 1  1  1  1  1  1  1  1  1  1  1  
	

In [18]:
scan2InputNN(nn2HiddenLayer32,weights=weights2Layers,showHidden=5,xysup=11)

Showing hidden neuron  5
      -5  -4  -3  -2  -1  0  1  2  3  4  5  
	5  1  1  1  1  1  1  1  1  1  1  1  
	4  1  1  1  1  1  1  1  1  1  1  1  
	3  -  1  1  1  1  1  1  1  1  1  1  
	2  -  -  1  1  1  1  1  1  1  1  1  
	1  -  -  -  -  1  1  1  1  1  1  1  
	0  -  -  -  -  -  1  1  1  1  1  1  
	-1 -  -  -  -  -  -  -  1  1  1  1  
	-2 -  -  -  -  -  -  -  -  1  1  1  
	-3 -  -  -  -  -  -  -  -  -  -  1  
	-4 -  -  -  -  -  -  -  -  -  -  -  
	-5 -  -  -  -  -  -  -  -  -  -  -  
	

# Generalizing the Perceptron

We have seen that we can understand the classical perceptron as a 2-step process, (1) taking an average and (2) labeling that average among two classes (the nonlinear step).

We also claimed that any nonlinearity will help us do the job. We want to see now that also any "aggregator" function will do.
<pre>
   X1 o  ___       __
       \(   )___  |  ____o 
       /(___)   __|      
   X2 o
   
The general aggregator function, also called predictor function. This simply takes N inputs (here shown only two)
and outputs a value. The latter may not necessarily be neither linear, nor continous nor even a deterministic 
function of the N inputs X1,...XN. This value is then eventually feeded into the non-linear activation function.
</pre>



In [122]:
def synapticPotentialWeighted(X,W):
    return np.dot(X,W[:len(X)])
    
def synapticPotentialAverage(X,W=None):
    return sum(X)/len(X)
    
def synapticPotentialMedian(X,W=None):
    """
    To make sense, 
    WT := \sum |W_i| := total # of X values (not all distinct) seen.
    Assing each X_i to its weight W_i -> (X_i,W_i)
    sort by X_i
    lx2=floor(WT/2)
    if exists j ; |W_j| = lx2 :
        md = ( W_j*X_j + W_(j+1) *X_(j+1) ) / ( W_j + W_(j+1)) #weighted average of both points
    if WT%2==0: 
    else:
        md
        
    Example: w1=7 w2=5 w3=2 => WT=14 => lx2=7 => md = (w1*x1+w2*x2 ) / (w1+w2) = (7x1+5x2)/12
                  
    if lx%2 == 0 ):
        md=(Z[lx2-1] + Z[lx2] )*0.5
    else:
        md=Z[lx2]*1.0 #so that it always returns a float
    """
    
    # W=[1,1,...,1] as many 1's as length of X
    if W == None: 
        W=np.ones(len(X)) 

    #Given array [x1,x2,...xn] -> get the sum_i xi
    def foldr(_X,_accu=0):          
        if len(_X) == 0: return _accu
        if len(_X) == 1: return _accu+_X[0]
        return (_accu + _X[-1] + foldr(_X[:-1]) )

    WT = foldr( list( map(abs,W) ) )
    
    def pfilter(par,x):
        for a,b in par:
            if abs(b) == x: return par.index((a,b))
        return None

    Z=sorted(list(zip(X,W)))
    lx2=floor(WT/2)
    #on Z=sorted(X,W): Python will mutate arguments **if it can**
               
    #if WT%2 == 0:
    #    j = pfilter(Z,lx2)
    #    try: return (Z[j][1]*Z[j][0]+Z[j+1][1]*Z[j+1][0]) 
    #    except: print('ERROR j= ',j," Z: ",Z, " WT: ",WT)
    #else:
    j=0 ; zw=abs(Z[j][1])
    while zw < lx2:
        j+=1
        zw+=abs(Z[j][1]) 
    if j == len(Z)-1: j-=1
    
    try: return (Z[j+1][1]*Z[j+1][0]+Z[j][1]*Z[j][0])/(abs(Z[j+1][1])+abs(Z[j][1])) # {wj*xj + w(j+1)*x(j+1)}/(wj+wj+1) 
    except: print('ERROR j=',j," Z: ",Z, " WT: ",WT)

def synapticPotentialMode(X,W):
    """
        To make sense, the weights W should be non-negative.
    Choose randomly one among all possible maxima
    
    When W[1,1] Provides a soft boundary between both phases (classes) with difussion across the boundary
    """
    wmax = max(list(map(abs,W)) )
    imax = [ i for i,j in enumerate(list(map(abs,W)) ) if j==wmax ]
    #print("max weights:",imax)
    i = np.random.choice(imax,1)
    return X[i]*W[i]/wmax
    

In [54]:
b[:2],synapticPotentialMedian(b[:2],[-10000,1]),synapticPotentialWeighted(b[:2],[1,1]),synapticPotentialAverage(b[:2])


([5, 7], -4.998800119988001, 12, 6.0)

In [54]:
b[:2],synapticPotentialMedian(b[:2],[-10000,1]),synapticPotentialWeighted(b[:2],[1,1]),synapticPotentialAverage(b[:2])


([5, 7], -4.998800119988001, 12, 6.0)

In [54]:
b[:2],synapticPotentialMedian(b[:2],[-10000,1]),synapticPotentialWeighted(b[:2],[1,1]),synapticPotentialAverage(b[:2])


([5, 7], -4.998800119988001, 12, 6.0)

In [54]:
b[:2],synapticPotentialMedian(b[:2],[-10000,1]),synapticPotentialWeighted(b[:2],[1,1]),synapticPotentialAverage(b[:2])


([5, 7], -4.998800119988001, 12, 6.0)

In [75]:
def nGenPerceptron(X,W,
                   synapticPotential=synapticPotentialMode,
                   activation=nonLinearBipolarStep,
                   #activation=(lambda z,s: nonLinearBipolarStep(nonLinearRelu(z,None),s)),
                   string=True,debug=False,showHidden=False):
    assert len(X)>1 , len(W)>1 
    
    return activation(synapticPotential(X,W),string)
    

In [123]:
scan2InputNN(nGenPerceptron,weights=[0,-1],xysup=3)


      -1  0  1  
	1  -  -  -  
	0  1  1  1  
	-1 1  1  1  
	

/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:74: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


# The bias or "external field"

This is a more detail model of a neuron
![detailed diagram neuron wikipedia](https://upload.wikimedia.org/wikipedia/commons/a/a9/Complete_neuron_cell_diagram_en.svg)

Connections between a neuron A and another B are established from the axon terminals of A to the dendrites of B (see on the left orange branches and red ones). It seems reasonable to think that the neuron is capable of "doing" something else with all the inputs it gets than just "summing that up".

We could think of enlarging our simple perceptron model by giving it a kind of "cell body".

Let's say the neuron doesn't just weight its inputs but diverts part of them to each other. In the following drawing, the weighted input from neuron 2 is split into two parts and one of them is added to the weighted contribution of neuron 1. Finally, a layer of two hidden neurons re-weights those two braches and the neuron decides whether to fire or not (or say to fire either +1 or -1) down its axon to the connection it's making with other neurons.

<pre>

X1 o
    \z1  x1*z1+&#937;
     \ _________            <!-- due to unicode char, drawing _seems_ misaligned. However renders well -->
      |         \z3          _
     &#937;|          ----------_| ------o y
      |_________/z4   
     /  x2*z2-&#937;
    /z2 
X2 o
</pre>

Let's call this new perceptron the **L-perceptron**, from the loop circuit this enlargement represents. 

We have then the the U-perceptron mixes up part of the inputs before thresholding. In this case, it diverts an amount of $\Omega$ from one input to the other. On the upper branch we then have $z_1\cdot x_1\,+\,\Omega$ while the lower branch carries $z_2\cdot x_2\,-\,\Omega$. Finally, the U-perceptron will fire a value $y$ depending on the aggregation signal $z\,\equiv\,z_3\cdot z_1\cdot x_1\,+\,+\,z_4\cdot z_2\cdot x_2\,+\,\Omega\,(z_3\,-\,z_4)$.

Let's introduce the following variables
$$w_1\,=\,z_3\,z_1 \\
  w_2\,=\,z_4\,z_2 \\
  b  \,=\,\Omega\,(z3-z4)
$$

Then the U-perceptron aggregator amounts to $$z\,=\,w_1\, x_1\,+\,w_2\, x_2\,+\,b\,$$

We see that the L-perceptron is simply our previous perceptron with an added *bias*, $b$ !! 

In the physics community, the bias $b$ is called the external field. In the context of modeling magnetic materials, the external field could be the external magnetic field acting on the sample material. 

From what we have seen so far, it is easy to realize that the bias introduces an additional source of signal before the neuron decides to fire +1 or -1. The L-perceptron is no longer a passive element, but actively contributes to the incoming signal. 

More specifically, the bias allows us to draw dividing lines that do not cross the origin (0,0) in a simple way. 

# Convex Hull

From this point on we will asume our perceptron is an L-perceptron, i.e., it comes with a bias $b$.

Let's see how we can use the perceptron to classify a convex region in the plane.

<pre>
         |
      ___|1___
 ____|___|____|______
   -2|___|____|2
       -1|
         |
      

</pre>

We can geneate four classifying lines with perpendiculars and biases given by L1: (0,1),1  L2:

#### Test pad

In [65]:
z=map((lambda x: x*x),a)
z= list(z)

def foldr(_X,_accu=0):
    """
    foldR
    """
    if len(_X) == 0: return _accu
    if len(_X) == 1: return _accu+_X[0]
    return (_accu + _X[-1] + foldr(_X[:-1]) )



list(z),foldr(z),foldr(a)

([1, 36, 9, 49, 4], 99, 19)

In [111]:
a=[1,-2,3]
c=list(map(abs,a))
np.dot(a,2)
a

[1, -2, 3]

In [66]:
abs(-1)

1

In [67]:
z

[1, 36, 9, 49, 4]

In [71]:
p=list(zip(a,z))
p,sorted(p)

([(1, 1), (6, 36), (3, 9), (7, 49), (2, 4)],
 [(1, 1), (2, 4), (3, 9), (6, 36), (7, 49)])

In [97]:
def pfilter(par,x):
    for a,b in par:
        if b == x: return par.index((a,b))
    return None

j=pfilter(p,490)
if j: print(j)
else: print('no')

no


In [94]:
p[3]

(7, 49)

In [43]:
a=[1,2,3,4]
sum(a)

10